In [ ]:
from abc import ABC, abstractmethod
from datetime import datetime
import uuid

# Person Class
class Person:
    def __init__(self, name, address, phone):
        self._name = name
        self._address = address
        self._phone = phone
    
    def get_details(self):
        return {
            "Name": self._name,
            "Address": self._address,
            "Phone": self._phone
        }

# Abstract Account Class
class Account(ABC):
    def __init__(self, owner, balance=0.0):
        self._account_number = str(uuid.uuid4())
        self._balance = balance
        self._owner = owner
        self._transaction_history = []

    @abstractmethod
    def deposit(self, amount):
        pass

    @abstractmethod
    def withdraw(self, amount):
        pass

    def get_balance(self):
        return self._balance

    def show_account_details(self):
        return {
            "Account Number": self._account_number,
            "Balance": self._balance,
            "Owner": self._owner.get_details()
        }

# Customer Class (inherits from Person)
class Customer(Person):
    def __init__(self, name, address, phone):
        super().__init__(name, address, phone)
        self.accounts = []

    def add_account(self, account):
        self.accounts.append(account)

    def get_accounts_summary(self):
        summary = [account.show_account_details() for account in self.accounts]
        return summary

# SavingsAccount Class (inherits from Account)
class SavingsAccount(Account):
    def __init__(self, owner, balance=0.0, interest_rate=0.03):
        super().__init__(owner, balance)
        self._interest_rate = interest_rate

    def deposit(self, amount):
        if amount > 0:
            self._balance += amount
            self._transaction_history.append(Transaction("Deposit", amount, self._balance))
            print(f"Deposited ${amount} to SavingsAccount. New Balance: Rs.{self._balance}")
        else:
            print("Invalid deposit amount.")

    def withdraw(self, amount):
        if 0 < amount <= self._balance:
            self._balance -= amount
            self._transaction_history.append(Transaction("Withdrawal", amount, self._balance))
            print(f"Withdrew ${amount} from SavingsAccount. New Balance: Rs.{self._balance}")
        else:
            print("Insufficient funds or invalid amount.")

    def apply_interest(self):
        interest = self._balance * self._interest_rate
        self._balance += interest
        print(f"Applied interest. New Balance: Rs.{self._balance}")

# BusinessAccount Class (inherits from Account)
class BusinessAccount(Account):
    def __init__(self, owner, balance=0.0, overdraft_limit=500.0):
        super().__init__(owner, balance)
        self._overdraft_limit = overdraft_limit

    def deposit(self, amount):
        if amount > 0:
            self._balance += amount
            self._transaction_history.append(Transaction("Deposit", amount, self._balance))
            print(f"Deposited ${amount} to BusinessAccount. New Balance: Rs.{self._balance}")
        else:
            print("Invalid deposit amount.")

    def withdraw(self, amount):
        if amount <= self._balance + self._overdraft_limit:
            self._balance -= amount
            self._transaction_history.append(Transaction("Withdrawal", amount, self._balance))
            print(f"Withdrew ${amount} from BusinessAccount. New Balance: Rs.{self._balance}")
        else:
            print("Overdraft limit exceeded.")

# Transaction Class
class Transaction:
    def __init__(self, transaction_type, amount, balance_after_transaction):
        self.transaction_id = str(uuid.uuid4())
        self.transaction_type = transaction_type
        self.amount = amount
        self.date = datetime.now()
        self.balance_after_transaction = balance_after_transaction

    def get_transaction_details(self):
        return {
            "Transaction ID": self.transaction_id,
            "Type": self.transaction_type,
            "Amount": self.amount,
            "Date": self.date,
            "Balance After Transaction": self.balance_after_transaction
        }

# Bank Class
class Bank:
    def __init__(self):
        self.customers = []

    def create_customer_account(self, name, address, phone, account_type, **kwargs):
        customer = Customer(name, address, phone)
        if account_type == "Savings":
            account = SavingsAccount(customer, kwargs.get('balance', 0.0), kwargs.get('interest_rate', 0.03))
        elif account_type == "Business":
            account = BusinessAccount(customer, kwargs.get('balance', 0.0), kwargs.get('overdraft_limit', 500.0))
        else:
            print("Invalid account type.")
            return None
        customer.add_account(account)
        self.customers.append(customer)
        print(f"{account_type} account created for {name} with account number {account._account_number}")
        return account

    def find_account(self, account_number):
        for customer in self.customers:
            for account in customer.accounts:
                if account._account_number == account_number:
                    return account
        return None

    def view_transaction_history(self, account):
        for transaction in account._transaction_history:
            print(transaction.get_transaction_details())

# Main Menu Function
def main_menu():
    bank = Bank()
    while True:
        print("\n--- Banking System Menu ---")
        print("1. Create Account")
        print("2. Deposit Money")
        print("3. Withdraw Money")
        print("4. Check Balance")
        print("5. View Transaction History")
        print("6. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            name = input("Enter customer name: ")
            address = input("Enter address: ")
            phone = input("Enter phone number: ")
            account_type = input("Enter account type (Savings/Business): ")
            balance = float(input("Enter initial deposit amount: "))
            if account_type == "Savings":
                interest_rate = float(input("Enter interest rate: "))
                bank.create_customer_account(name, address, phone, account_type, balance=balance, interest_rate=interest_rate)
            elif account_type == "Business":
                overdraft_limit = float(input("Enter overdraft limit: "))
                bank.create_customer_account(name, address, phone, account_type, balance=balance, overdraft_limit=overdraft_limit)
            else:
                print("Invalid account type.")

        elif choice == '2':
            account_number = input("Enter account number: ")
            account = bank.find_account(account_number)
            if account:
                amount = float(input("Enter deposit amount: "))
                account.deposit(amount)
            else:
                print("Account not found.")

        elif choice == '3':
            account_number = input("Enter account number: ")
            account = bank.find_account(account_number)
            if account:
                amount = float(input("Enter withdrawal amount: "))
                account.withdraw(amount)
            else:
                print("Account not found.")

        elif choice == '4':
            account_number = input("Enter account number: ")
            account = bank.find_account(account_number)
            if account:
                print("Current Balance:", account.get_balance())
            else:
                print("Account not found.")

        elif choice == '5':
            account_number = input("Enter account number: ")
            account = bank.find_account(account_number)
            if account:
                bank.view_transaction_history(account)
            else:
                print("Account not found.")

        elif choice == '6':
            print("Exiting the system. Goodbye!")
            break

        else:
            print("Invalid choice. Please try again.")

# Run the main menu
main_menu()



--- Banking System Menu ---
1. Create Account
2. Deposit Money
3. Withdraw Money
4. Check Balance
5. View Transaction History
6. Exit
Savings account created for Varun with account number d8418651-902b-4dde-9310-6e866b17ca0f

--- Banking System Menu ---
1. Create Account
2. Deposit Money
3. Withdraw Money
4. Check Balance
5. View Transaction History
6. Exit
Deposited $620.0 to SavingsAccount. New Balance: $1240.0

--- Banking System Menu ---
1. Create Account
2. Deposit Money
3. Withdraw Money
4. Check Balance
5. View Transaction History
6. Exit
Current Balance: 1240.0

--- Banking System Menu ---
1. Create Account
2. Deposit Money
3. Withdraw Money
4. Check Balance
5. View Transaction History
6. Exit
Current Balance: 1240.0

--- Banking System Menu ---
1. Create Account
2. Deposit Money
3. Withdraw Money
4. Check Balance
5. View Transaction History
6. Exit
Current Balance: 1240.0

--- Banking System Menu ---
1. Create Account
2. Deposit Money
3. Withdraw Money
4. Check Balance
5. Vie